# Prepare

I will prepare my data for exploreation in this notebook.

**Imports**

In [1]:
# general imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# internal imports
import sys
sys.path.append("/Users/migashane/CodeUp/Data_Science/telco_churn_classification_project/util_")
import acquire_

**Get data**

The data here is comming from `acquire_.py` file in my `util_` folder. I see have thhe query here if closure is neede on exact data retrieved.

In [2]:
# get my data
telco, query = acquire_.get_telco_data()

telco.head().T # transpose my data to get a better look

,0,1,2,3,4
payment_type_id,2,2,1,1,2
internet_service_type_id,1,1,2,2,2
contract_type_id,2,1,1,1,1
customer_id,0002-ORFBO,0003-MKNFE,0004-TLHLJ,0011-IGKFF,0013-EXCHZ
gender,Female,Male,Male,Male,Female
senior_citizen,0,0,0,1,1
partner,Yes,No,No,Yes,Yes
dependents,Yes,No,No,No,No
tenure,9,9,4,13,3
phone_service,Yes,Yes,Yes,Yes,Yes


## Remove redundant columns

In [3]:
# run a cross tabulation on internet type and internet type id...
# to see if I don't have duplicated imformation
pd.crosstab(telco.internet_service_type, telco.internet_service_type_id)

internet_service_type_id,1,2,3
internet_service_type,,,
DSL,2421,0,0
Fiber optic,0,3096,0
None,0,0,1526


In [4]:
# run an other cross tabulation on contract_type and contract_type_id...
# to see if I don't have duplicated imformation
pd.crosstab(telco.contract_type_id, telco.contract_type)

contract_type,Month-to-month,One year,Two year
contract_type_id,,,
1,3875,0,0
2,0,1473,0
3,0,0,1695


In [5]:
# run an other cross tabulation on payment_type and payment_type_id...
# to see if I don't have duplicated imformation
pd.crosstab(telco.payment_type, telco.payment_type_id)

payment_type_id,1,2,3,4
payment_type,,,,
Bank transfer (automatic),0,0,1544,0
Credit card (automatic),0,0,0,1522
Electronic check,2365,0,0,0
Mailed check,0,1612,0,0


In [6]:
# lis of columns to remove
drop_cols = ["customer_id", 
             "internet_service_type_id", 
             "contract_type_id",
            "payment_type_id"]

# remove the columns
print("Original dataframe size:", telco.shape)
telco = telco.drop(columns=drop_cols)
print("New dataframe size:", telco.shape)

Original dataframe size: (7043, 24)
New dataframe size: (7043, 20)


**Removed columns:**

- `custromer_id` doesn't look important in predicting churn, so I will remove it from my dataframe

- `internet_service_type_id` and `internet_service_type` are relaying the same information, so I will remove internet_service_type_id beause it is valuable for me to know the name of the type of internet.

- `contract_type` and `contract_type_id` are also relaying the same information, so I will remove contract_type_id because it is important that I know the name of the contract.

- `payment_type` and `payment_type_id` are also relaying the same information, so I will remove payment_type_id because it is important that I know the name of the type of payment.

# Check columns datatypes

look at what is contained in the columns and find incorect datatypes.

**Object columns**

In [7]:
# retrieve object data type columns
telco.select_dtypes("object").head(3)

,gender,partner,dependents,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,total_charges,churn,contract_type,internet_service_type,payment_type
0,Female,Yes,Yes,Yes,No,No,Yes,No,Yes,Yes,No,Yes,593.3,No,One year,DSL,Mailed check
1,Male,No,No,Yes,Yes,No,No,No,No,No,Yes,No,542.4,No,Month-to-month,DSL,Mailed check
2,Male,No,No,Yes,No,No,No,Yes,No,No,No,Yes,280.85,Yes,Month-to-month,Fiber optic,Electronic check


In [8]:
# loock at the unique values stored in the total charges column
telco.total_charges.value_counts(dropna=False)

           11
20.2       11
19.75       9
19.9        8
20.05       8
           ..
2387.75     1
6302.8      1
2058.5      1
829.55      1
3707.6      1
Name: total_charges, Length: 6531, dtype: int64

In [20]:
# get the mean of all the rows of total charges that are actual numbers
mean_of_digits_in_total_charges = telco[telco.total_charges.str.isdigit()].total_charges.astype("float").mean()


# replace all the empty (" ") cells with the mean of the digit rows of total_charges
# then convert the column into a float data type column
telco["total_charges"] = telco["total_charges"].str.replace(" ", str(mean_of_digits_in_total_charges)).astype("float")

# check if the column was coverted succesfult by retrieving object data type columns again
telco.select_dtypes("object").head(3)

,gender,partner,dependents,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,churn,contract_type,internet_service_type,payment_type
0,Female,Yes,Yes,Yes,No,No,Yes,No,Yes,Yes,No,Yes,No,One year,DSL,Mailed check
1,Male,No,No,Yes,Yes,No,No,No,No,No,Yes,No,No,Month-to-month,DSL,Mailed check
2,Male,No,No,Yes,No,No,No,Yes,No,No,No,Yes,Yes,Month-to-month,Fiber optic,Electronic check


**Numericcolumns**

In [22]:
telco.select_dtypes("number").head(3)

,senior_citizen,tenure,monthly_charges,total_charges
0,0,9,65.6,593.30
1,0,9,59.9,542.40
2,0,4,73.9,280.85


**Check data types:**

- total_charges` column is a string buy that is not the correect data type, so I will covert it into float.

    - This column contains 11 rows filled by empty spaces (" ") into them, so I will replace them with `mean` of the the column.
   

## Visualize values counts

**Numeric columns**

In [25]:
numrics_cols = telco.select_dtypes("number")
